# Get autoregressive predictions from pretrained checkpoint

In [ ]:
%cd ..

In [ ]:
from maskpredformer.scheduled_sampling_trainer import MaskSimVPScheduledSamplingModule
from maskpredformer.vis_utils import show_gif, show_video_line

import random
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
ckpt_path = "checkpoints/simvp_ss_epoch=2-valid_last_frame_iou=0.456.ckpt"
module = MaskSimVPScheduledSamplingModule.load_from_checkpoint(ckpt_path, use_gt_data=True, unlabeled=False)

In [ ]:
@torch.no_grad()
def get_result(i):
    x, y= module.val_set[i]
    x = x.unsqueeze(0).to(module.device)
    y = y.unsqueeze(0).to(module.device)
    cur_seq = module.sample_autoregressive(x, 11)
    x = x.squeeze(0).cpu().numpy()
    y = y.squeeze(0).cpu().numpy()
    y_hat = cur_seq.squeeze(0).cpu().numpy()
    return x, y, y_hat

In [ ]:
def get_all_results():
    all_yhat = []
    for i in tqdm(range(len(module.val_set))):
        x, y, y_hat = get_result(i)
        all_yhat.append(y_hat)
    return np.stack(all_yhat)

In [ ]:
all_yhat = get_all_results()


In [ ]:
root_val_dir = "data/val_gt/"
gt_masks = np.stack([np.load(os.path.join(root_val_dir, f"video_{i}", "mask.npy")) for i in range(1000, 2000)])

# Visualization

In [ ]:
def get_gif(i):
    x, y, y_hat = get_result(i)
    show_gif(x, y, y_hat, out_path='./result.gif')
get_gif(50)

In [ ]:
show_video_line(gt_masks[50], 22)

In [ ]:
show_video_line(np.concatenate((gt_masks[50, :11], all_yhat[50]), axis=0), 22)

In [ ]:
# show_video_line(np.concatenate([all_x[vis_idx], all_yhat[vis_idx]], axis=0), 22)

# IoU calculation

In [ ]:
from torchmetrics import JaccardIndex

In [ ]:
jaccard = JaccardIndex(task='multiclass', num_classes=49)

In [ ]:
jaccard(torch.from_numpy(all_yhat[:,-1]), torch.from_numpy(gt_masks[:, -1]))